In [49]:
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [50]:
with open("pose_data.json") as f:
    data = json.load(f)

In [51]:
vid_index = 0
frame_index = 0
data[vid_index]['keypoints'][frame_index]

[[736.3848876953125, 302.6104431152344, 0.9935799837112427],
 [763.4105834960938, 269.35498046875, 0.9908037185668945],
 [722.4285888671875, 281.6174011230469, 0.9510730504989624],
 [841.2077026367188, 273.429931640625, 0.9530590772628784],
 [737.1574096679688, 295.5009765625, 0.3484953045845032],
 [942.5046997070312, 429.872314453125, 0.9874888062477112],
 [728.5203247070312, 457.90234375, 0.984792947769165],
 [986.2774658203125, 644.3109130859375, 0.9289509057998657],
 [701.4364013671875, 657.3375854492188, 0.8985481858253479],
 [938.4482421875, 862.135986328125, 0.8444055914878845],
 [652.78955078125, 810.4200439453125, 0.7793729901313782],
 [876.927001953125, 869.8792114257812, 0.7474165558815002],
 [727.307373046875, 869.4553833007812, 0.7418520450592041],
 [866.533935546875, 896.0, 0.03009786084294319],
 [661.199951171875, 896.0, 0.02973119355738163],
 [845.7664184570312, 896.0, 0.002895983401685953],
 [653.7989501953125, 896.0, 0.003010145854204893]]

In [56]:
def get_velocity(keypooints):
    '''
    calculate the velocity by subtracting frame by previous frame
    :param data:
    :param vid_index:
    :return:
    '''
    velocities = []
    for frame_num in range(1, len(keypoints[vid_index])):
        this_frame = np.array(keypoints[vid_index][frame_num])[:2]
        next_frame = np.array(keypoints[vid_index][frame_num - 1])[:2]
        velocity = this_frame - next_frame
        velocities.append(velocity)
    return np.array(velocities)

keypoints = data[vid_index]['keypoints']
get_velocity(keypoints)

array([[  27.0256958 ,  -33.25546265],
       [ -40.98199463,   12.26242065],
       [ 118.77911377,   -8.18746948],
       [-104.05029297,   22.07104492],
       [ 205.34729004,  134.37133789],
       [-213.984375  ,   28.0300293 ],
       [ 257.75714111,  186.40856934],
       [-284.84106445,   13.02667236],
       [ 237.01184082,  204.79840088],
       [-285.65869141,  -51.71594238],
       [ 224.13745117,   59.45916748],
       [-149.61962891,   -0.42382812],
       [ 139.2265625 ,   26.5446167 ],
       [-205.33398438,    0.        ],
       [ 184.56646729,    0.        ],
       [-191.96746826,    0.        ]])

In [58]:
def get_positions(keypoints):
    positions = []
    for frame_num in range(1, len(keypoints[vid_index])):
        position = np.array(keypoints[vid_index][frame_num])[:2]
        positions.append(position)
    return np.array(positions)

get_positions(keypoints)

array([[763.4105835 , 269.35498047],
       [722.42858887, 281.61740112],
       [841.20770264, 273.42993164],
       [737.15740967, 295.50097656],
       [942.50469971, 429.87231445],
       [728.52032471, 457.90234375],
       [986.27746582, 644.31091309],
       [701.43640137, 657.33758545],
       [938.44824219, 862.13598633],
       [652.78955078, 810.42004395],
       [876.92700195, 869.87921143],
       [727.30737305, 869.4553833 ],
       [866.53393555, 896.        ],
       [661.19995117, 896.        ],
       [845.76641846, 896.        ],
       [653.7989502 , 896.        ]])

In [5]:
data[0][1]['keypoints'][0][:2]

[856.4679565429688, 534.758544921875]

In [59]:
def normalize(data):
    vel = data.astype(float).copy()
    max_xy = np.abs(vel).max(axis=tuple(range(vel.ndim - 1)), keepdims=True)
    max_xy = np.maximum(max_xy, 1e-8)
    return vel / max_xy

In [62]:
def get_pos_vel(keypoints):
    pos = get_positions(keypoints)  # shape (..., Dp)
    vel = get_velocity(keypoints)   # shape (..., Dv)
    return np.concatenate([pos, vel], axis=-1)

In [64]:
pos_val = get_pos_vel(keypoints)
normalize(pos_val)

array([[ 0.77403227,  0.3006194 ,  0.09460834, -0.16238146],
       [ 0.73248007,  0.31430514, -0.1434649 ,  0.05987557],
       [ 0.85291181,  0.30516733,  0.41580781, -0.03997819],
       [ 0.74741382,  0.3298002 , -0.3642469 ,  0.10776962],
       [ 0.9556182 ,  0.47976821,  0.71885539,  0.65611517],
       [ 0.73865656,  0.51105172, -0.74909107,  0.13686645],
       [ 1.        ,  0.719097  ,  0.90232557,  0.9102052 ],
       [ 0.71119581,  0.7336357 , -0.99713775,  0.0636073 ],
       [ 0.95150531,  0.96220534,  0.82970289,  1.        ],
       [ 0.66187211,  0.90448666, -1.        , -0.25252122],
       [ 0.88912809,  0.97084733,  0.78463375,  0.29033023],
       [ 0.73742674,  0.97037431, -0.52377062, -0.00206949],
       [ 0.87859042,  1.        ,  0.4873878 ,  0.1296134 ],
       [ 0.67039953,  1.        , -0.71880881,  0.        ],
       [ 0.85753396,  1.        ,  0.64610836,  0.        ],
       [ 0.66289556,  1.        , -0.6720169 ,  0.        ]])

# Training Stuff

In [68]:
data[0]

{'vid_path': 'Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_1.mp4',
 'gesture_id': 0,
 'keypoints': [[[736.3848876953125, 302.6104431152344, 0.9935799837112427],
   [763.4105834960938, 269.35498046875, 0.9908037185668945],
   [722.4285888671875, 281.6174011230469, 0.9510730504989624],
   [841.2077026367188, 273.429931640625, 0.9530590772628784],
   [737.1574096679688, 295.5009765625, 0.3484953045845032],
   [942.5046997070312, 429.872314453125, 0.9874888062477112],
   [728.5203247070312, 457.90234375, 0.984792947769165],
   [986.2774658203125, 644.3109130859375, 0.9289509057998657],
   [701.4364013671875, 657.3375854492188, 0.8985481858253479],
   [938.4482421875, 862.135986328125, 0.8444055914878845],
   [652.78955078125, 810.4200439453125, 0.7793729901313782],
   [876.927001953125, 869.8792114257812, 0.7474165558815002],
   [727.307373046875, 869.4553833007812, 0.7418520450592041],
   [866.533935546875, 896.0, 0.03009786084294319],
   [661.199951171875, 896.0, 0.0297311935

In [69]:
# gesture, vid, frame
for video_entry in data:
    path = video_entry['vid_path']
    gesture = video_entry['gesture_id']
    keypoints = video_entry['keypoints']
    frames = len(keypoints)
    print(f"gesture: {gesture}, \tframes: {frames},  \tpath: {path}")




gesture: 0, 	frames: 30,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_1.mp4
gesture: 0, 	frames: 30,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_3.mp4
gesture: 0, 	frames: 35,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_2.mp4
gesture: 0, 	frames: 15,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_6.mp4
gesture: 0, 	frames: 17,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_5.mp4
gesture: 0, 	frames: 18,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_4.mp4
gesture: 1, 	frames: 65,  	path: Images/small/look_at_me/look_at_me_Subject1_Tuan_cam-1-index_1.mp4
gesture: 1, 	frames: 66,  	path: Images/small/look_at_me/look_at_me_Subject1_Tuan_cam-1-index_3.mp4
gesture: 1, 	frames: 0,  	path: Images/small/look_at_me/look_at_me_Subject1_Tuan_cam-1-index_2.mp4
gesture: 1, 	frames: 39,  	path: Images/small/look_at_me/look_at_me_Subject1_Tuan_cam-1-index_6.mp4
gesture: 1, 	frames: 36, 

In [72]:
# https://www.geeksforgeeks.org/python/filter-in-python/

def more_than_one_frame(video_entry):
    keypoints = video_entry['keypoints']
    frames = len(keypoints)

    return frames > 1

filtered = list(filter(more_than_one_frame, data))

max_vid_length = max(len(kp) for kp in filtered)
print(max_vid_length)

for video_entry in filtered:
    path = video_entry['vid_path']
    gesture = video_entry['gesture_id']
    keypoints = video_entry['keypoints']
    frames = len(keypoints)
    print(f"gesture: {gesture}, \tframes: {frames},  \tpath: {path}")

3
gesture: 0, 	frames: 30,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_1.mp4
gesture: 0, 	frames: 30,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_3.mp4
gesture: 0, 	frames: 35,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_2.mp4
gesture: 0, 	frames: 15,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_6.mp4
gesture: 0, 	frames: 17,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_5.mp4
gesture: 0, 	frames: 18,  	path: Images/small/peekaboo/peekaboo_Subject1_Tuan_cam-1-index_4.mp4
gesture: 1, 	frames: 65,  	path: Images/small/look_at_me/look_at_me_Subject1_Tuan_cam-1-index_1.mp4
gesture: 1, 	frames: 66,  	path: Images/small/look_at_me/look_at_me_Subject1_Tuan_cam-1-index_3.mp4
gesture: 1, 	frames: 39,  	path: Images/small/look_at_me/look_at_me_Subject1_Tuan_cam-1-index_6.mp4
gesture: 1, 	frames: 36,  	path: Images/small/look_at_me/look_at_me_Subject1_Tuan_cam-1-index_5.mp4
gesture: 1, 	frames: 4